In [112]:
import pandas as pd
from PIL import Image
import base64
import io

# data = pd.read_csv("./yuchang_data/MMMU-1023-MC-2238.tsv", sep="\t")
# data = pd.read_csv("./yuchang_data/SEEDBench-1000-MC-4.tsv", sep="\t")
# data = pd.read_csv("/pasteur2/u/suyc/VLMEval/VLMEvalKit/LMUData/VMCBench.tsv", sep="\t").to_dict(orient="records")
# len(data)

data = pd.read_csv("./VMCBench.tsv", sep="\t").to_dict(orient="records")
print(len(data))

def base64_to_image(base64_str):
    """
    Convert a base64 string to a PIL Image.
    
    Args:
        base64_str (str): The base64 encoded image string.
        
    Returns:
        PIL.Image.Image: The image object.
    """
    # Decode the base64 string into bytes
    image_data = base64.b64decode(base64_str)
    
    # Convert bytes into a PIL image
    image = Image.open(io.BytesIO(image_data))
    
    return image

900


In [ ]:
# # concat all the *50.tsv* in /pasteur2/u/suyc/VLMEval/VLMEvalKit/LMUData into a single tsv

# import glob as glob
# import os

# data = []

# for f in glob.glob("/pasteur2/u/suyc/VLMEval/VLMEvalKit/LMUData/*50.tsv"):
#     subset = pd.read_csv(f, sep="\t").to_dict(orient="records")
#     for item in subset:
#         item["source"] = os.path.basename(f)
#     data += subset
# len(data)

# # pd.DataFrame(data).to_csv("VMCBench.tsv", sep="\t", index=False)

{'index': 35429,
 'question': "what is this bus route's number?",
 'A': '38',
 'B': '23',
 'C': '33',
 'D': '133',
 'answer': 'C',
 'source': 'TextVQA_VAL-500-MC-50.tsv',
 'multiple_choice_answer': '33',
 'choices': nan,
 'correct_choice_idx': nan}

In [128]:
from copy import deepcopy

idx = 58

item = deepcopy(data[idx])
image = base64_to_image(item["image"])
item.pop('image')
display(image)
display(item)


In [ ]:
from openai import OpenAI
import base64
from pydantic import BaseModel
from textwrap import dedent


class Judgement(BaseModel):
    correct: bool
    explanation: str


client = OpenAI(timeout=20)
def judge_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
    """
    Judge whether the multiple-choice question is correct based on choices and the image (provided as base64).

    Args:
    image_base64 (str): Base64 encoded image string.
    choices (list): A list of distractor answers.
    correct_choice (str): The correct answer choice.

    Returns:
    str: GPT-4's judgment on whether the question is correct.
    """
    
    # # Construct the prompt for GPT-4o
    # prompt = f"""
    # You are tasked with determining whether a multiple-choice question has clearly one correct answer based on the provided image and choices.
    
    # The image has been encoded in base64 format. Below are the choices and the correct answer.
    
    # Question: {question}
    # Correct Answer: {correct_choice}
    # Distractor Choices: {", ".join(choices)}
    
    # Please analyze the image and determine if the question is correct. Correctness is defined as there being one clearly correct answer, with the choices representing different concepts.
    # If you cannot understand the image or answer the question, please provide a judgment based on the choices only to see if choices are different enough.

    # Provide a yes/no answer and explain in at most 2 sentences.
    # """

    # Construct the prompt for GPT-4o
    prompt = f"""
        Analyze the following multiple-choice question choices and correct answer:

        Choices: {choices}
        Correct Answer: {correct_choice}

        Task: Determine if the correct answer is mutually exclusive with all other choices. This means it should be impossible for the correct answer and any other choice to both be true simultaneously.

        Respond with:
        1. "Yes" if the correct answer is mutually exclusive with all other choices, or "No" if there's any possibility of overlap between the correct answer and any other choice.
        2. A brief explanation (max 2 sentences) justifying your decision. If answering "No", identify which choice(s) could potentially overlap with the correct answer.
    """

    # Assume the API supports image input alongside text (modify based on actual model/API capabilities)
    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",  # Adjust this to the appropriate multimodal model ID if needed
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                # {
                #     "type": "image_url",
                #     "image_url": {
                #         "url": f"data:image/jpeg;base64,{image_base64}"
                #     }
                # }
            ]}
        ],
        response_format=Judgement,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    # Extract and return the assistant's response
    answer = response.choices[0].message.parsed
    return answer



In [ ]:
import pandas as pd
import numpy as np
import os

model = "cambrian_8b"
for filename in os.listdir(f"./yuchang_records_new/{model}"):
    if "SEEDBench" not in filename:
        continue
    data = pd.read_excel(f"./yuchang_records_new/{model}/{filename}").to_dict("records")
    accs = []
    for item in data:
        answer = item["answer"]
        # pred = item["prediction"][0]
        # accs.append(answer == pred)
        pred = item["prediction"]
        accs.append(f" {answer}" in pred)

    acc = np.mean(accs)
    print(f"{filename}: {acc}")

In [ ]:
import pandas as pd
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

model = "cambrian_8b"
# data = pd.read_csv("./yuchang_data/MMMU-1023-MC-2238.tsv", sep="\t").to_dict("records")
# data1 = pd.read_excel(f"./yuchang_records_new/{model}/MMMU-1023_mc.xlsx").to_dict("records")
# data2 = pd.read_excel(f"./yuchang_records_new/{model}/MMMU-1023-MC-1403_mc.xlsx").to_dict("records")
data = pd.read_csv("./yuchang_data/SEEDBench-1000-MC-4.tsv", sep="\t").to_dict("records")
data1 = pd.read_excel(f"./yuchang_records_new/{model}/SEEDBench-1000-MC_mc.xlsx").to_dict("records")
data2 = pd.read_excel(f"./yuchang_records_new/{model}/SEEDBench-1000-MC-OLD_mc.xlsx").to_dict("records")
print(len(data), len(data1), len(data2))

def is_correct(answer, prediction):
    # return answer == prediction
    return f" {answer}" in prediction

corrects1 = [is_correct(item["answer"], item["prediction"]) for item in data1]
corrects2 = [is_correct(item["answer"], item["prediction"]) for item in data2]


# for i in range(10):
#     item = data[i]
#     image_base64 = item["image"]
#     distractors = [item["A"], item["B"], item["C"], item["D"]]
#     correct_choice = item["answer"]
#     judgment = judge_multichoice_correctness_with_image(image_base64, distractors, correct_choice)
#     print(judgment)


for idx in range(len(data1)):
    if not corrects2[idx] and corrects1[idx]:
        item1 = data1[idx]
        item2 = data2[idx]
        item = data[idx]
        image_base64 = item["image"]
        image_data = base64.b64decode(image_base64)
        image = Image.open(BytesIO(image_data)).resize((256, 256))

        # if "What kind of sugar is this, aldose or ketose?" not in item["question"]: continue

        assert item1["question"] == item2["question"] == item["question"]
        display(image)
        print(item1["question"])
        print([item1["A"], item1["B"], item1["C"], item1["D"]], item1["answer"], item1["prediction"])
        print([item2["A"], item2["B"], item2["C"], item2["D"]], item2["answer"], item2["prediction"])
        # print(data2[idx])
        # print(data1[idx])

        item = item1
        question = item["question"]
        choices = [item["A"], item["B"], item["C"], item["D"]] # [:item['choice_length']]
        correct_choice = item[item["answer"]]
        print(question, choices, correct_choice)
        judgment = judge_multichoice_correctness_with_image(image_base64, question, choices, correct_choice)
        print(judgment)

        item = item2
        question = item["question"]
        choices = [item["A"], item["B"], item["C"], item["D"]]
        correct_choice = item[item["answer"]]
        print(question, choices, correct_choice)
        judgment = judge_multichoice_correctness_with_image(image_base64, question, choices, correct_choice)
        print(judgment)


In [ ]:
from openai import OpenAI
import base64
from pydantic import BaseModel
from textwrap import dedent


class Judgement(BaseModel):
    correct: bool
    explanation: str


client = OpenAI(timeout=20)
def judge_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
    """
    Judge whether the multiple-choice question is correct based on choices and the image (provided as base64).

    Args:
    image_base64 (str): Base64 encoded image string.
    choices (list): A list of distractor answers.
    correct_choice (str): The correct answer choice.

    Returns:
    str: GPT-4's judgment on whether the question is correct.
    """
    
    # # Construct the prompt for GPT-4o
    # prompt = f"""
    # You are tasked with determining whether a multiple-choice question has clearly one correct answer based on the provided image and choices.
    
    # The image has been encoded in base64 format. Below are the choices and the correct answer.
    
    # Question: {question}
    # Correct Answer: {correct_choice}
    # Distractor Choices: {", ".join(choices)}
    
    # Please analyze the image and determine if the question is correct. Correctness is defined as there being one clearly correct answer, with the choices representing different concepts.
    # If you cannot understand the image or answer the question, please provide a judgment based on the choices only to see if choices are different enough.

    # Provide a yes/no answer and explain in at most 2 sentences.
    # """

    # Construct the prompt for GPT-4o
    prompt = f"""
        Analyze the following multiple-choice question choices and correct answer:

        Choices: {choices}
        Correct Answer: {correct_choice}

        Task: Determine if the correct answer is mutually exclusive with all other choices. This means it should be impossible for the correct answer and any other choice to both be true simultaneously.

        Respond with:
        1. "Yes" if the correct answer is mutually exclusive with all other choices, or "No" if there's any possibility of overlap between the correct answer and any other choice.
        2. A brief explanation (max 2 sentences) justifying your decision. If answering "No", identify which choice(s) could potentially overlap with the correct answer.
    """

    # Assume the API supports image input alongside text (modify based on actual model/API capabilities)
    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",  # Adjust this to the appropriate multimodal model ID if needed
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                # {
                #     "type": "image_url",
                #     "image_url": {
                #         "url": f"data:image/jpeg;base64,{image_base64}"
                #     }
                # }
            ]}
        ],
        response_format=Judgement,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    # Extract and return the assistant's response
    answer = response.choices[0].message.parsed
    return answer

